## Imports & Setups

In [5]:
from transformers import pipeline
import torch
from datasets import load_dataset
from tqdm import tqdm
import json
import numpy as np
from typing import List

## SFT and Constitution Model (mimic)

In [2]:
# used to mimic the SL model and the constitution model
# see: https://huggingface.co/stabilityai/stablelm-2-zephyr-1_6b
pipe_SL = pipeline("text-generation", model="stabilityai/stablelm-2-zephyr-1_6b")
pipe_constitution = pipeline("text-generation", model="stabilityai/stablelm-2-zephyr-1_6b")

# wrapper function for the pipeline
def retrieve_pipe_response(pipe, prompt, max_new_tokens=32, temperature=None, num_return_sequences=1):
    """
    Given a prompt, return the response from the pipeline
    :param pipe: the pipeline to use
    :param prompt: the prompt to use to the LM
    :param max_new_tokens: the maximum number of tokens to generate
    :param temperature: the temperature to use for sampling
    :param num_return_sequences: the number of sequences to return
    :return res(list): a list of responses, length = num_return_sequences
    """
    do_sample = True if temperature is not None else False
    responses = pipe(prompt,
                     max_new_tokens=max_new_tokens,
                     temperature=temperature,
                     do_sample=do_sample,
                     num_return_sequences=num_return_sequences)
    res = []
    for response in responses:
        res.append(response['generated_text'][len(prompt):].strip())
    return res

## Data: Harmful prompts 

In this section we save the harmful prompts (prompts only) from anthropic/hh-rlhf dataset.

data source: [anthropic/hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf)

In [3]:
# Load one of the harmless subsets
dataset = load_dataset("Anthropic/hh-rlhf", data_dir="harmless-base", cache_dir="/scratch/students/haolli")

In [4]:
# Parsing & saving the harmful prompts
harmful_prompts = []
for i, sample in tqdm(enumerate(dataset['train'])):
    conversation = sample['chosen']
    harmful_prompt = conversation.split("\n\n")[1].split("Human: ")[-1]
    harmful_prompts.append(harmful_prompt)
for i, sample in tqdm(enumerate(dataset['test'])):
    conversation = sample['chosen']
    harmful_prompt = conversation.split("\n\n")[1].split("Human: ")[-1]
    harmful_prompts.append(harmful_prompt)

with open('/scratch/students/haolli/harmful_prompts_anthropic_hh.json', 'w') as file:
    json.dump(harmful_prompts, file)

42537it [00:01, 24082.20it/s]
2312it [00:00, 28662.39it/s]


## Experimental Setup

In this section we will curate 3 datasets based on 3 moral perspectives (Action, Motivation, Consequence). 


### Retrieving Response from the SFT Model
For each harmful prompts generated above, we will ask the SFT model twice, each ask in details, we ask (in a chain):
- The model's answer (__Action__);
- The model's motivation behind the answer (__Motivation__);
- The utility of the action towards promoting a virtuous character in the agent (__Consequence__)

In this sense, we have a chain of responses (**P**rompt -> **A**ction -> **M**otivation -> **C**onsequence).

Since for each prompt, we ask twice, we have:
- P -> A1 -> M1 -> C1
- P -> A2 -> M2 -> C2

### Curating Datasets for RM Training

We will have a list of virtues for the Constitutional Model to refer to.

For each column in the response pair (i.e. A1&A2, M1&M2, C1&C2), we ask the Constitional Model to rank the pair based on each virtue.

For each response pair, we end up with something like P | A1 | [virtue_scores] (could be converted to a scalar score by avg. pooling).

Since we have 3 kinds of moral perspectives (A, M, C), we will end up with 3 datasets.

### Virtue Definition & Prompt to Constitutional AI

The Constituional AI will judge on the paired response to generate a vector of scores based on the virtues list.

The virtues & relevant prompts to inject into the Constitutional AI are defined as follows

In [8]:
virtues = ["Which of the following assistant responses is more fair?",
           "Which of the following assistant responses is more just?"
             ]

In [7]:
def generate_prompt_to_constitution(virtue: str, harmful_prompt_to_SFT_model: str, SFT_model_response: List[str]) -> str:
    """
    Generate the prompt for the constitution model.
    :param virtue: the virtue to be considered, it's one of the elements in the virtues list
    :param harmful_prompt_to_SFT_model: the harmful prompt to the SFT model (human query)
    :param SFT_model_response: the two responses from the SFT model based on the harmful prompt
    :return prompt_to_constitution: the prompt for the constitution model
    """
    
    prompt_to_constitution = f"""
    Consider the following conversation between a human and an assistant:
    Human: {harmful_prompt_to_SFT_model}
    {virtue}
    ---- Start of assistant response options ----
    Options:
    (A){SFT_model_response[0]}
    (B){SFT_model_response[1]}
    ---- End of assistant response options ----
    Your response must be in JSON format, for example, like this:
    {{
    "choice": "A" or "B",
    }}
    """
    return prompt_to_constitution

### Generate the AMC response dataset

In [13]:
# exp
import random
harmful_prompts_exp = random.choices(harmful_prompts, k=10)

# TODO: finish AMC curation
for harmful_prompt in harmful_prompts_exp:
    SFT_model_response_Action = retrieve_pipe_response(pipe_SL, harmful_prompt, temperature=0.1, num_return_sequences=2)
    ...
    for virtue in virtues:
        ...
        # prompt_to_constitution = generate_prompt_to_constitution(virtue, harmful_prompt, SFT_model_response)
        # constitution_response = retrieve_pipe_response(pipe_constitution, prompt_to_constitution, temperature=0.1, num_return_sequences=1)
        # print(prompt_to_constitution)
        # print(constitution_response)

Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.


KeyboardInterrupt: 